In [3]:
import pandas as pd
from arcgis.gis import GIS

C:\Users\rami8629\AppData\Local\ESRI\conda\envs\arcgispro-py3-geopandas\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.1)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [5]:
# Define the Excel file name and sheet names
excel_file_path = r"D:\OneDrive - Esri\Demos & Blogs\Code-Repo\Notebooks\ArcGIS_Online_Management.xlsx"
user_sheet_name = 'inactive_items_per_user'
item_sheet_name = 'inactive_items'

# Read the user and item data from Excel
user_df = pd.read_excel(excel_file_path, sheet_name=user_sheet_name)
item_df = pd.read_excel(excel_file_path, sheet_name=item_sheet_name)

# Initialize the ArcGIS GIS object (you may need to provide your ArcGIS Online organization URL and credentials)
gis = GIS("Pro")

In [15]:
all_items

{'ae42b498eb6d4604bc3af064d0c9b6b0', '28fe3bd88c5444408ade7dd0d7ae627c', 'c587553694da4f46a0c9c3a6cd1e81d6', '84e2a140ad96401783c777373de2e4e0', '26d1cede121843fcb1d02206afeac3bd'}

In [46]:
# Function to get a list of item IDs owned by a user and check if they are deprecated
def get_and_check_items(username):
    user = gis.users.get(username)
    
    deprecated_items = []
    valid_items = []

    users_folder_list = user.folders
    items_root = user.items()
    for item in items_root:
        item_row = item_df[item_df['item_id'] == item.id]
        if not item_row.empty and item_row['keep_item_yes_no'].values[0] == 'No':
            deprecated_items.append(item.id)
        else:
            valid_items.append(item.id)
    for user_folder in users_folder_list:
        for item in user.items(folder=user_folder['title']):
            item_row = item_df[item_df['item_id'] == item.id]
            if not item_row.empty and item_row['keep_item_yes_no'].values[0] == 'No':
                deprecated_items.append(item.id)
            else:
                valid_items.append(item.id)

    return deprecated_items, valid_items

# Function to unshare and delete deprecated items
def process_deprecated_items(deprecated_item_ids):
    for item_id in deprecated_item_ids:
        item = gis.content.get(item_id)
        
        # Unshare the item
        item.share(everyone=False, org=False, groups=[])
        
        # Delete the item
        item.delete()
        print(f"Item: {item['name']} of id:{item['id']} owned by {item['owner']} is deleted")

# Function to change ownership of valid items and delete the previous owner
def change_owner_and_delete_previous_owner(valid_item_ids, new_owner_username):
    for item_id in valid_item_ids:
        item = gis.content.get(item_id)
        
        # Change ownership to the new owner
        item.reassign_to(new_owner_username)
        
        # Delete the previous owner (specified in user_df)
    try:
        user_to_delete = gis.users.get(item.owner)
        user_to_delete.delete()
        print(f"{item.owner} is deleted from ArcGIS Online")
    except Exception as e:
        print(f"Error deleting previous owner {item.owner}: {str(e)}")


In [44]:
deprecated_items

[]

In [45]:
valid_items

70

In [47]:
# Initialize a list to store deprecated item IDs and valid item IDs
all_deprecated_items = []
all_valid_items = []

# Iterate through the user DataFrame and get/deprecate items
for _, row in user_df.iterrows():
    username = row['username']
    deprecated_items, valid_items = get_and_check_items(username)
    
    all_deprecated_items.extend(deprecated_items)
    all_valid_items.extend(valid_items)

# Remove duplicates from the deprecated_items and valid_items lists
all_deprecated_items = list(set(all_deprecated_items))
all_valid_items = list(set(all_valid_items))

# Print the results
print("All Deprecated Items:")
print(all_deprecated_items)

print("\nAll Valid Items:")
print(all_valid_items)

All Deprecated Items:
[]

All Valid Items:
['32dc439091424382ad32f441fefc7e5f', '1ed849316e974ad0b7bd0cb72a9dddc1', 'f8384d03a61d4014a485c61aeb238419', 'a0f092dde3aa43a1baa276b45ee9dbf9', '43574fa867554f0aa6556099dc1322d2', 'cd85827e08a141f599cb31b3edfd761a', '7aec71b8aa774bb0a687c74d3aa37d14', '1a4e142d0bee4d00a90cac48e7936a2a', 'abbe69b747f24eaca715760f38931842', '7dfe8468a362406b8b6fcc6e8f31a5a0', '5f282570c0bf4cb0907d13e5e6d8c154', '60236d0d981a4edcb0e03cdb9f5a978c', 'a5d61d59ac4a450884f84ea344c0f409', 'f9603e57590047e7b473264ca26b6bce', '7d69a178a11e4a1782fa76ce70ea4cef', 'd485e6542df6468eaa9f2ce17f53b6de', 'c3c0e96846b3487c8af0bfafd420547f', '1927508dd0f145b4a63bf295aa5c887d', '2b6a4cd1b3bd4b11a1f3f9bb5cd9922a', 'c3620becf01d44128850c4991866e122', '658c3a2cc1b74882b58b7e7c7b2265c5', '9b7cf9f7f49f4af2abac8ceefe73222b', '8a643cf28c604920ab1723d60d9e5dd9', 'ac8b329013ec467bb0d740a1b62c2a94', '7788503cd2d44377b65f2e7e752d2721', '0bf4904c10874f9a871eda735f7a1554', 'e9e393944cd24e219ef

In [34]:
# Process deprecated items (unshare and delete)
process_deprecated_items(all_deprecated_items)

# Change ownership of valid items to the signed-in user
signed_in_user = gis.users.me
change_owner(all_valid_items, signed_in_user.username)

Item: 93rd_St_Elem.PNG of id:ae42b498eb6d4604bc3af064d0c9b6b0 owned by ksmyth_aid is deleted
Item: NLL_BA_Demo.docx of id:28fe3bd88c5444408ade7dd0d7ae627c owned by ksmyth_aid is deleted
Item: None of id:c587553694da4f46a0c9c3a6cd1e81d6 owned by ksmyth_aid is deleted
Item: None of id:84e2a140ad96401783c777373de2e4e0 owned by ksmyth_aid is deleted
Item: None of id:26d1cede121843fcb1d02206afeac3bd owned by ksmyth_aid is deleted
